# Implementation of the Paper:
# Paper Implementation – EfficientNet-B0 on CIFAR-10

This notebook contains the implementation of the original EfficientNet-B0 model using PyTorch. The goal is to evaluate the **baseline performance** of the pretrained model on a new dataset — CIFAR-10 — without any fine-tuning. This step allows us to measure how well the model generalizes to new data and serves as a reference point for later improvements.



# 1. Importing Required Libraries
We import necessary PyTorch modules, torchvision datasets and models, and utilities for data loading. EfficientNet-B0 is loaded directly from `torchvision.models`, which provides a pretrained version trained on the ImageNet dataset.


# 2. Loading the Pretrained Model
We load the pretrained EfficientNet-B0 model with `weights='IMAGENET1K_V1'`, meaning it uses weights trained on ImageNet (1000 classes). Since CIFAR-10 only has 10 classes, we replace the final fully connected (classifier) layer with a new `Linear` layer that outputs 10 classes.

This allows the model architecture to match the output of the CIFAR-10 dataset.

```python
model = efficientnet_b0(weights='IMAGENET1K_V1')
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 10)



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# torchvision provides efficientnet_b0/b1, 
from torchvision.models import efficientnet_b0, efficientnet_b1


# We load the CIFAR-10 dataset using torchvision.datasets. The dataset contains 60,000 32×32 RGB images divided into 10 classes. Standard preprocessing steps are applied:

-Random cropping and flipping (only to training set) for data augmentation

-Normalization using CIFAR-10 dataset statistics

In [2]:

# Prepare the CIFAR-10 dataset and Dataloaders

# Common transforms for CIFAR-10
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # mean
                         (0.2470, 0.2435, 0.2616))  # std
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

# Download/Load CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)

# Create train & test loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)



100%|██████████| 170M/170M [00:02<00:00, 69.0MB/s] 


In [3]:

# Load Pretrained Model

# Let's pick EfficientNet B0 
model = efficientnet_b0(weights='IMAGENET1K_V1')  # loads pretrained weights

# We need to replace the final classification layer to have 10 outputs for CIFAR-10.
num_ftrs = model.classifier[1].in_features  # typically 1280 for EfficientNet-B0
model.classifier[1] = nn.Linear(num_ftrs, 10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\selvi/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 52.9MB/s]


In [4]:
# Check the model architecture:
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

# Evaluating the Pretrained Model (Before Fine-Tuning):
We define an evaluate() function to measure accuracy. This function:

Sets the model to evaluation mode

Passes all test images through the model

Compares predicted labels to actual labels

Calculates and returns overall test accuracy

Since we haven’t fine-tuned the model yet, this shows us how well the pretrained ImageNet weights perform directly on CIFAR-10.

In [5]:
#  Evaluate performance BEFORE fine-tuning
# we want to see how the pretrained model does on CIFAR-10 as-is, run a quick validation pass:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    return accuracy

pre_finetune_acc = evaluate(model, test_loader, device)
print(f"Accuracy of the pretrained (unfine-tuned) model on CIFAR-10: {pre_finetune_acc:.2f}%")


Accuracy of the pretrained (unfine-tuned) model on CIFAR-10: 8.50%


# This notebook replicates the original EfficientNet-B0 model, modifies its output layer to suit CIFAR-10, and evaluates it without any training.